In [1]:
%pip install -q chromadb
%pip install chromadb open-clip-torch
%pip install ipywidgets

import os
import uuid
import chromadb
from chromadb.utils.data_loaders import ImageLoader
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction

In [ ]:
# Step 1 - Setup Chroma Client

def chroma_client_in_memory() -> chromadb.Client:
    """
    สร้าง Chroma client ในโหมดฝังในโปรเซส (embedded)
    — เก็บในหน่วยความจำ (in-memory) โดยดีฟอลต์ => ข้อมูลหายเมื่อโปรเซส/โน้ตบุ๊กปิดหรือรีสตาร์ท
    """
    return chromadb.Client()

def chroma_client_persistent(persist_directory: str = "./chroma_db") -> chromadb.PersistentClient:
    """
    สร้าง Chroma client ในโหมดฝังในโปรเซส (embedded)
    — เก็บแบบถาวร (persistent) ในไดเรกทอรีที่ระบุ
    - ควรเรียกใช้ client.persist() เมื่อมีการแก้ไขข้อมูล (เพิ่ม ลบ อัปเดต) เพื่อการันตีการบันทึกการเปลี่ยนแปลงลงดิสก์
    """

    return chromadb.PersistentClient(path=persist_directory)

def chroma_client_http() -> chromadb.HttpClient:
    """
    Run chroma server options
    1. cmd: chroma run
    2. docker compose → compose.yml

    Returns:
        chromadb.HttpClient: 
        - to check connection: telnet 127.0.0.1 8000
    """
    return chromadb.HttpClient(host="localhost", port=8000)

# client = chroma_client_in_memory()
client = chroma_client_persistent("./chroma_multimodal_db")
# client = chroma_client_http() 

In [ ]:
# Step 2 - สร้าง Collection แบบ multimodal ด้วย OpenCLIPEmbeddingFunction
# https://docs.trychroma.com/docs/embeddings/multimodal#adding-multimodal-data-and-data-loaders
collection = client.get_or_create_collection(
    name="multimodal_collection",
    embedding_function=chromadb.utils.embedding_functions.OpenCLIPEmbeddingFunction(),
    data_loader=ImageLoader(), # Optional: สำหรับโหลดและแปลง "ภาพ" เป็น embeddings
    metadata={"hnsw:space": "cosine"}   # Optional: กำหนด space สำหรับ HNSW indexing ด้วย cosine similarity
)

In [4]:
# Step 3 - เพิ่ม Data เข้า collection
all_image_paths = [ "images/dog001.png", "images/dog002.png", "images/dog003.png",
                    "images/cat001.png", "images/cat002.png", "images/cat003.png",
                ]
labels = [  "dog", "dog", "dog",
            "cat", "cat", "cat",
        ]
doc_ids = [str(uuid.uuid4()) for _ in all_image_paths]

# Prepare metadata สำหรับเก็บข้อมูลรูปภาพ
metadatas = [
    {
        "label": lbl,
        "filename": os.path.basename(path),
        "description": f"A photo of a {lbl}"
    }
    for lbl, path in zip(labels, all_image_paths)
]

collection.add(
    ids=doc_ids,
    uris=all_image_paths, # <--- ป้อนแค่ Path ของไฟล์รูปภาพได้เลย เดี๋ยว ImageLoader จะไปโหลดและแปลงเป็น embeddings ให้เอง
    metadatas=metadatas
)

In [5]:
# Step 4 - ทดสอบ Query ด้วย ข้อความ (text queries)

query_text = "dog"
res = collection.query(
    query_texts=[query_text], # <--- ค้นหาด้วยคำว่า "dog"
    n_results=4,
    include=["metadatas", "distances"]
)

# แสดงผลลัพธ์
print(f"\\n=== Query: '{query_text}' ===")
for rank, (meta, dist) in enumerate(zip(res["metadatas"][0], res["distances"][0]), start=1):
    print(f"#{rank} -> {meta['filename']} ({meta['label']}) distance={dist:.4f}")

\n=== Query: 'dog' ===
#1 -> dog003.png (dog) distance=0.7122
#2 -> dog001.png (dog) distance=0.7355
#3 -> dog002.png (dog) distance=0.7449
#4 -> cat002.png (cat) distance=0.7775


In [6]:
# Step 5 - ทดสอบ Query ด้วย รูปภาพ (image queries)

query_image_path = "images/cat001.png" # <--- ใช้รูปภาพ "cat001.png" ในการ query

# The ImageLoader, which was set up with the collection, will handle loading
# and embedding the image from its URI.
res_image = collection.query(
    query_uris=[query_image_path],
    n_results=4,
    include=["metadatas", "distances"]
)

# แสดงผลลัพธ์
print(f"\n=== Query with image: '{os.path.basename(query_image_path)}' ===")
for rank, (meta, dist) in enumerate(zip(res_image["metadatas"][0], res_image["distances"][0]), start=1):
    print(f"#{rank} -> {meta['filename']} ({meta['label']}) distance={dist:.4f}")


=== Query with image: 'cat001.png' ===
#1 -> cat001.png (cat) distance=0.0000
#2 -> cat002.png (cat) distance=0.2716
#3 -> cat003.png (cat) distance=0.3282
#4 -> dog003.png (dog) distance=0.4329


### Query โดยใช้ Metadata

เราสามารถคัดกรองการค้นหาข้อมูลใน Collection โดยระบุเงื่อนไขคัดกรองจาก metadata ที่ป้อนไว้ได้ โดยใช้พารามิเตอร์ `where` ในเมธอด `query`

In [9]:
# ค้นหาเฉพาะรูปภาพที่มี 'label' เป็น 'dog'
res_metadata_query = collection.query(
    query_texts=[""], # ใส่ query_texts เอาไว้เพื่อให้ผ่านเงื่อนไขของ ChromaDB api
    where={"label": "dog"},
    n_results=3,
    include=["metadatas", "uris"]
)

print("\n=== Query by metadata (label = 'dog') ===")
for rank, meta in enumerate(res_metadata_query["metadatas"][0], start=1):
    print(f"#{rank} -> {meta['filename']} ({meta['label']})")

# ค้นหาด้วยเงื่อนไขที่ซับซ้อนขึ้น เช่น 'label' เป็น 'cat' และ 'filename' มีคำว่า '002'
res_complex_metadata_query = collection.query(
    query_texts=[""], # ใส่ query_texts เอาไว้เพื่อให้ผ่านเงื่อนไขของ ChromaDB api
    where={"$and": [
        {"label": "cat"},
        {"filename": "cat002.png"} # เปลี่ยนจาก $contains เป็น exact match
    ]},
    n_results=1,
    include=["metadatas", "uris"]
)

print("\n=== Query by complex metadata (label = 'cat' AND filename is 'cat002.png') ===")
if res_complex_metadata_query["metadatas"] and res_complex_metadata_query["metadatas"][0]:
    for rank, meta in enumerate(res_complex_metadata_query["metadatas"][0], start=1):
        print(f"#{rank} -> {meta['filename']} ({meta['label']})")
else:
    print("No results found for complex metadata query.")


=== Query by metadata (label = 'dog') ===
#1 -> dog003.png (dog)
#2 -> dog002.png (dog)
#3 -> dog001.png (dog)

=== Query by complex metadata (label = 'cat' AND filename is 'cat002.png') ===
#1 -> cat002.png (cat)
